In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
import pathlib
import os
from transformers import AutoProcessor, GroundingDinoForObjectDetection
from PIL import Image
import torch
import shutil  # To move images to another folder
import gc  # Garbage collection for memory management
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import shutil
import time

In [ ]:
!unzip drive/MyDrive/images.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_tickfont.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_outlinewidth.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_len.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_showticklabels.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_tickformat.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_title.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_showtickprefix.py  
  inflating: images/.venv/lib/python3.9/site-packages/plotly/validators/scattergeo/marker/colorbar/_ypad.py  
  inflating: images/.venv/lib/p

In [ ]:
for i in os.listdir("images"):
  if i not in [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"] :
    print(i)
    os.rename("images/"+i ,"images/"+i.replace("_", " ").replace("%2c", " ").replace("-", " ") )


shoes
cardboard_waste
white-glass
single_use_plastic_cutlery_spoon%2c_fork%2c_straw%2c_dish%2c_glass
green-glass
clear_glass_juice%2c_perfume%2c_glass
zero_shot_object_detection_with_grounding_dino.ipynb
news_paper_waste
trash
battery
bottel
aluminum_foil_kitchen
paper
plastic
cardboard
metal
aerosol_waste_cans
clothes
brown-glass
biological
pomogranade_peel
pizza_box
pizza_box_output


In [ ]:
class TrainAutoModel:
    def __init__(self):
        self.folders = []
        self.processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-base")
        self.model = GroundingDinoForObjectDetection.from_pretrained("IDEA-Research/grounding-dino-base").to('cpu')  # GPU
        self.boxes = []
        self.scores = []

    def list_of_folders(self):
        return [f for f in pathlib.Path("images").iterdir() if f.is_dir()]

    def list_of_files(self, file):
        return [f for f in pathlib.Path("images/"+file).iterdir() if f.is_file()]

    def folder_creation(self):
        self.folders = self.list_of_folders()
        for i in self.folders:
            label_name = i.name  # Get folder name as string
            if not os.path.exists(f"trainImages/{label_name}"):
                os.makedirs(f"trainImages/{label_name}/images")
                os.makedirs(f"trainImages/{label_name}/labels")

    def model_train(self, path, text, label_name, threshold=0.35):
        image = Image.open(path).convert('RGB')
        image = image.resize((640, 480))  # Resize to reduce memory
        inputs = self.processor(images=image, text=text, return_tensors="pt").to('cpu')
        outputs = self.model(**inputs)
        target_sizes = torch.tensor([image.size[::-1]]).to('cpu')  # Image size in (H, W) format
        results = self.processor.image_processor.post_process_object_detection(outputs, threshold=threshold, target_sizes=target_sizes)[0]

        detected = False
        img_w, img_h = image.size

        if len(results['scores']) > 0:  # Check if there are any detections
            detected = True
            label_file_content = []  # Store lines for the YOLOv8 format
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                if score >= threshold:
                    box = box.tolist()
                    self.boxes.append(box)
                    self.scores.append(score.tolist())
                    print(f"Detected {label.item()} with confidence {score} at location {box}")

                    # Generate YOLOv8 format coordinates
                    label_file_content.append(self.generate_yolo_format(box, img_w, img_h, label))

            # Move image and save all detections into one label file
            self.move_image_and_save_label(path, label_file_content, label_name)

        # Clean up large variables and run garbage collection after each image is processed
        del image, inputs, outputs, target_sizes, results
        torch.cuda.empty_cache()  # Clear GPU memory
        gc.collect()  # Run garbage collection

        return detected

    def generate_yolo_format(self, box, img_w, img_h, label):
        # Calculate YOLOv8 format (normalized center_x, center_y, width, height)
        x_min, y_min, x_max, y_max = box
        x_center = (x_min + x_max) / 2.0 / img_w
        y_center = (y_min + y_max) / 2.0 / img_h
        width = (x_max - x_min) / img_w
        height = (y_max - y_min) / img_h

        # Assuming label class ID is `0` for now, can be changed based on your use case
        class_id = 0  # Change this to appropriate class mapping if needed
        return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

    def move_image_and_save_label(self, path, label_file_content, label_name):
        # Create destination directories
        dest_dir_images = f"trainImages/{label_name}/images"
        dest_dir_labels = f"trainImages/{label_name}/labels"
        os.makedirs(dest_dir_images, exist_ok=True)
        os.makedirs(dest_dir_labels, exist_ok=True)

        # Move the image
        image_name = pathlib.Path(path).name
        dest_image_path = f"{dest_dir_images}/{image_name}"
        shutil.move(path, dest_image_path)
        label_file_path = " "

        if image_name.split(".")[-1] == "jpg":
            print(image_name)
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.jpg', '.txt')}"
            print(label_file_content)
            with open(label_file_path, 'w') as f:
                for line in label_file_content:
                    f.write(line + "\n")
        elif image_name.split(".")[-1] == "png":
            print(image_name)
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.png', '.txt')}"
            print(label_file_content)
            with open(label_file_path, 'w') as f:
                for line in label_file_content:
                    f.write(line + "\n")
        else :
            print(image_name)
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.jpeg', '.txt')}"
            print(label_file_path)
            with open(label_file_path, 'w') as f:
                for line in label_file_content:
                    f.write(line + "\n")



        plt.show()



In [ ]:
original_name = sample = os.listdir("images")
originale_second = [i.replace("_"," ").replace("%2c" ," ").replace("-" ," ") for i in original_name]
print(originale_second)

for i , j in zip(original_name , originale_second):
  i = f"images/{i}"
  j = f"images/{j}"
  os.rename(i ,j )

In [ ]:
tm = TrainAutoModel()

# Get list of folders excluding specific ones
folder_list = [i for i in os.listdir("images") if i not in [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"]]
print(folder_list)

for folder in folder_list:
    files = tm.list_of_files(folder)
    print(folder, len(files))

    for file in files:
        print(file)
        if file not in [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"]:

            detected = tm.model_train(path=str(file), text=f"{folder}.", label_name=folder)
            if detected:
                print(f"{file} moved and labeled successfully.")
            else:
                print(f"No detection for {file}.")
            gc.collect()
            time.sleep(100)  # Additional collection after each image processing

    shutil.move(str(folder) ,"drive/MyDrive/train_images/" )

In [ ]:
os.makedirs("drive/MyDrive/train_images", exist_ok=True)

In [ ]:
shutil.move("sample_data" , "drive/MyDrive/train_images/")

In [ ]:
class TrainAutoModel:
    def __init__(self):
        self.folders = []
        self.processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-base")
        self.model = GroundingDinoForObjectDetection.from_pretrained("IDEA-Research/grounding-dino-base").to('cuda')  # GPU
        self.boxes = []
        self.scores = []
        self.excluded_files = [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"]
        self.trainImages = "trainImages"

    def list_of_folders(self):
        return [f for f in pathlib.Path("images").iterdir() if f.is_dir() ]

    def list_of_files(self, file):
        return [f for f in pathlib.Path("images/"+file).iterdir() if f.is_file()]

    def folder_creation(self):
        self.folders = self.list_of_folders()
        for i in self.folders:
            label_name = i.name  # Get folder name as string
            if not os.path.exists(f"{self.trainImages}/{label_name}"):
                os.makedirs(f"{self.trainImages}/{label_name}/images")
                os.makedirs(f"{self.trainImages}/{label_name}/labels")

    def model_train(self, path, text, label_name, threshold=0.35):
        image = Image.open(path).convert('RGB')
        image = image.resize((640, 480))  # Resize to reduce memory
        inputs = self.processor(images=image, text=text, return_tensors="pt").to('cuda')
        outputs = self.model(**inputs)
        target_sizes = torch.tensor([image.size[::-1]]).to('cuda')  # Image size in (H, W) format
        results = self.processor.image_processor.post_process_object_detection(outputs, threshold=threshold, target_sizes=target_sizes)[0]

        detected = False
        img_w, img_h = image.size

        if len(results['scores']) > 0:  # Check if there are any detections
            detected = True
            label_file_content = []  # Store lines for the YOLOv8 format
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                if score >= threshold:
                    box = box.tolist()
                    self.boxes.append(box)
                    self.scores.append(score.tolist())
                    print(f"Detected {label.item()} with confidence {score} at location {box}")

                    # Generate YOLOv8 format coordinates
                    label_file_content.append(self.generate_yolo_format(box, img_w, img_h, label))

            # Move image and save all detections into one label file
            self.move_image_and_save_label(path, label_file_content, label_name)

        # Clean up large variables and run garbage collection after each image is processed
        del image, inputs, outputs, target_sizes, results
        torch.cuda.empty_cache()  # Clear GPU memory
        gc.collect()  # Run garbage collection

        return detected

    def generate_yolo_format(self, box, img_w, img_h, label):
        # Calculate YOLOv8 format (normalized center_x, center_y, width, height)
        x_min, y_min, x_max, y_max = box
        x_center = (x_min + x_max) / 2.0 / img_w
        y_center = (y_min + y_max) / 2.0 / img_h
        width = (x_max - x_min) / img_w
        height = (y_max - y_min) / img_h

        # Assuming label class ID is `0` for now, can be changed based on your use case
        class_id = 0  # Change this to appropriate class mapping if needed
        return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
    def move_to_all(self , folder):
      os.makedirs("drive/MyDrive/train_images" , exist_ok=True)
      shutil.move(str(folder) ,"drive/MyDrive/train_images/")

    def move_image_and_save_label(self, path, label_file_content, label_name):
        # Create destination directories
        dest_dir_images = f"{self.trainImages}/{label_name}/images"
        dest_dir_labels = f"{self.trainImages}/{label_name}/labels"
        os.makedirs(dest_dir_images, exist_ok=True)
        os.makedirs(dest_dir_labels, exist_ok=True)

        # Move the image
        image_name = pathlib.Path(path).name
        dest_image_path = f"{dest_dir_images}/{image_name}"
        shutil.move(path, dest_image_path)
        label_file_path = " "
        # Write all detections to the YOLOv8 label file
        if image_name.split(".")[-1] == "jpg":
            print(image_name)
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.jpg', '.txt')}"
            print(label_file_content)
            with open(label_file_path, 'w') as f:
                for line in label_file_content:
                    f.write(line + "\n")
        elif image_name.split(".")[-1] == "png":
            print(image_name)
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.png', '.txt')}"
            print(label_file_content)
            with open(label_file_path, 'w') as f:
                for line in label_file_content:
                    f.write(line + "\n")
        else :
            print(image_name)
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.jpeg', '.txt')}"
            print(label_file_path)
            with open(label_file_path, 'w') as f:
                for line in label_file_content:
                    f.write(line + "\n")
        plt.show()

In [ ]:
tm = TrainAutoModel()

# Get list of folders excluding specific ones
folder_list = [i for i in os.listdir("images") if i not in [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"]]
print(folder_list)

for folder in folder_list:
    files = tm.list_of_files(folder)
    print(folder, len(files))

    for file in files:
      try :
          if str(file).split("/")[-1] not in [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"]:
            detected = tm.model_train(path=str(file), text=f"{folder}.",label_name=folder)
            if detected:
                print(f"{file} moved and labeled successfully.")
            else:
                print(f"No detection for {file}.")
            gc.collect()  # Additional collection after each image processing
          else :
            print("nothing done")
      except Exception as e :
        print("exception create at file" , e)
        with open("exception.txt" , "a") as f:
          f.write(f"exception image file {file} as {e}")
    time.sleep(100)  # Additional collection after each image processing
    tm.move_to_all(folder)


In [ ]:
import gc

gc.collect()

In [ ]:
class TrainAutoModel:
    def __init__(self):
        self.folders = []
        self.processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-base")
        self.model = GroundingDinoForObjectDetection.from_pretrained("IDEA-Research/grounding-dino-base").to('cuda')  # GPU
        self.boxes = []
        self.scores = []
        self.excluded_files = [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"]
        self.trainImages = "trainImages"

    def list_of_folders(self):
        return [f for f in pathlib.Path("images").iterdir() if f.is_dir() ]

    def list_of_files(self, file):
        return [f for f in pathlib.Path("images/"+file).iterdir() if f.is_file()]

    def folder_creation(self):
        self.folders = self.list_of_folders()
        for i in self.folders:
            label_name = i.name  # Get folder name as string
            if not os.path.exists(f"{self.trainImages}/{label_name}"):
                os.makedirs(f"{self.trainImages}/{label_name}/images")
                os.makedirs(f"{self.trainImages}/{label_name}/labels")

    def model_train(self, path, text, label_name, threshold=0.35):
        image = Image.open(path).convert('RGB')
        image = image.resize((640, 480))  # Resize to reduce memory
        inputs = self.processor(images=image, text=text, return_tensors="pt").to('cuda')
        outputs = self.model(**inputs)
        target_sizes = torch.tensor([image.size[::-1]]).to('cuda')  # Image size in (H, W) format
        results = self.processor.image_processor.post_process_object_detection(outputs, threshold=threshold, target_sizes=target_sizes)[0]

        detected = False
        img_w, img_h = image.size

        if len(results['scores']) > 0:  # Check if there are any detections
            detected = True
            label_file_content = []  # Store lines for the YOLOv8 format
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                if score >= threshold:
                    box = box.tolist()
                    self.boxes.append(box)
                    self.scores.append(score.tolist())
                    print(f"Detected {label.item()} with confidence {score} at location {box}")

                    # Generate YOLOv8 format coordinates
                    label_file_content.append(self.generate_yolo_format(box, img_w, img_h, label))

            # Move image and save all detections into one label file
            self.move_image_and_save_label(path, label_file_content, label_name)

        # Clean up large variables and run garbage collection after each image is processed
        del image, inputs, outputs, target_sizes, results
        torch.cuda.empty_cache()  # Clear GPU memory
        gc.collect()  # Run garbage collection

        return detected

    def generate_yolo_format(self, box, img_w, img_h, label):
        # Calculate YOLOv8 format (normalized center_x, center_y, width, height)
        x_min, y_min, x_max, y_max = box
        x_center = (x_min + x_max) / 2.0 / img_w
        y_center = (y_min + y_max) / 2.0 / img_h
        width = (x_max - x_min) / img_w
        height = (y_max - y_min) / img_h

        # Assuming label class ID is `0` for now, can be changed based on your use case
        class_id = 0  # Change this to appropriate class mapping if needed
        return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
    def move_to_all(self , folder):
      os.makedirs("drive/MyDrive/train_images" , exist_ok=True)
      shutil.move(f"{self.trainImages}/{str(folder)}" ,"drive/MyDrive/train_images/")

    def move_image_and_save_label(self, path, label_file_content, label_name):
        # Create destination directories
        dest_dir_images = f"{self.trainImages}/{label_name}/images"
        dest_dir_labels = f"{self.trainImages}/{label_name}/labels"
        os.makedirs(dest_dir_images, exist_ok=True)
        os.makedirs(dest_dir_labels, exist_ok=True)

        # Move the image
        image_name = pathlib.Path(path).name
        dest_image_path = f"{dest_dir_images}/{image_name}"
        shutil.move(path, dest_image_path)
        label_file_path = " "
        # Write all detections to the YOLOv8 label file
        if image_name.split(".")[-1] == "jpg":
            print(image_name)
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.jpg', '.txt')}"
            print(label_file_content)
            with open(label_file_path, 'w') as f:
                for line in label_file_content:
                    f.write(line + "\n")
        elif image_name.split(".")[-1] == "png":
            print(image_name)
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.png', '.txt')}"
            print(label_file_content)
            with open(label_file_path, 'w') as f:
                for line in label_file_content:
                    f.write(line + "\n")
        else :
            print(image_name)
            label_file_path = f"{dest_dir_labels}/{image_name.replace('.jpeg', '.txt')}"
            print(label_file_path)
            with open(label_file_path, 'w') as f:
                for line in label_file_content:
                    f.write(line + "\n")
        plt.show()

In [ ]:
tm = TrainAutoModel()

# Get list of folders excluding specific ones
folder_list = [i for i in os.listdir("images") if i not in [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"]]
print(folder_list)

for folder in folder_list:
    files = tm.list_of_files(folder)
    print(folder, len(files))

    for file in files:
      try :
          if str(file).split("/")[-1] not in [".ipynb_checkpoints", ".idea", ".venv", ".DS_Store"]:
            detected = tm.model_train(path=str(file), text=f"{folder}.",label_name=folder)
            if detected:
                print(f"{file} moved and labeled successfully.")
            else:
                print(f"No detection for {file}.")
            gc.collect()  # Additional collection after each image processing
          else :
            print("nothing done")
      except Exception as e :
        print("exception create at file" , e)
        with open("exception.txt" , "a") as f:
          f.write(f"exception image file {file} as {e}")
    time.sleep(100)  # Additional collection after each image processing
    tm.move_to_all(folder)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['cardboard waste', 'pizza box', 'news paper waste', 'biological', 'pomogranade peel', 'cardboard', 'plastic', 'white glass', 'single use plastic cutlery spoon  fork  straw  dish  glass', 'metal', 'shoes', 'aerosol waste cans', 'clear glass juice  perfume  glass', 'paper', 'pizza box output', 'aluminum foil kitchen', 'clothes', 'zero shot object detection with grounding dino.ipynb', 'green glass', 'trash']
cardboard waste 7991
Detected 2 with confidence 0.6491121649742126 at location [339.7892761230469, 130.36744689941406, 516.415283203125, 443.0023498535156]
Detected 2 with confidence 0.6340643763542175 at location [236.3853302001953, 128.919677734375, 382.6976013183594, 433.3855285644531]
Detected 2 with confidence 0.5415596961975098 at location [-0.0019383430480957031, 102.46355438232422, 70.67345428466797, 262.9763488769531]
Detected 2 with confidence 0.5960966348648071 at location [126.26887512207031, 126.44783020019531, 259.5504150390625, 414.6783752441406]
card_board_waste_box_i

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Detected 2 with confidence 0.5662521719932556 at location [0.052947998046875, 0.03859519958496094, 640.052978515625, 480.03857421875]
card_board_waste_box_image_944.png
['0 0.500083 0.500080 1.000000 1.000000']
images/cardboard waste/card_board_waste_box_image_944.png moved and labeled successfully.
Detected 2 with confidence 0.9414584040641785 at location [46.987571716308594, 105.29039001464844, 590.89013671875, 385.36126708984375]
card_board_waste_box_image_883.png
['0 0.498342 0.511095 0.849848 0.583481']
images/cardboard waste/card_board_waste_box_image_883.png moved and labeled successfully.
Detected 2 with confidence 0.600507378578186 at location [29.184341430664062, 123.2249755859375, 604.1698608398438, 479.8434753417969]
card_board_waste_box_image_2433.png
['0 0.494808 0.628196 0.898415 0.742955']
images/cardboard waste/card_board_waste_box_image_2433.png moved and labeled successfully.
Detected 2 with confidence 0.578603148460

In [ ]:
import shutil
shutil.rmtree("images/brown glass")